# L0. Introduction

### Define all the imports

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Change to main repo folder for the imports
_, dir = os.path.split(os.getcwd())
if dir == 'notebooks': 
    os.chdir('..')
    sys.path.append(os.getcwd())
# Ignore warnings from pandas
pd.set_option('mode.chained_assignment', None)

In [ ]:
from src.data.reader import Reader

### Define new Functions

In [ ]:
def build_timeseries(data,cols):
    timeseries = pd.DataFrame(data, columns=cols)
    timeseries['stamp'] = pd.to_datetime(timeseries['stamp'], unit='s')
    timeseries = timeseries.set_index('stamp')
    return timeseries

In [ ]:
def filter_static_landmarks(lm, barcodes):
    for L,l in dict(barcodes).items(): # Translate barcode num to landmark num
        lm[lm==l]=L
    lm = lm[lm.type > 5] # Keep only static landmarks 
    return lm 

The previous block is for defining specific notebook functions

### Load a dataset

In [ ]:
# Define dataset to read
dataset = "data/MRCLAM_Dataset1" # Dataset
end_frame = 50000 # Extension of the dataset
robot = 'Robot5' # Robot

# Reader. Go to the src directory using VScode and inspect the class methods
self = Reader(dataset, robot, end_frame)

### Generate some representations

In [ ]:
# Ground truth data
plt.plot(self.groundtruth_data[:, 1], self.groundtruth_data[:, 2], 'b', label="Robot State Ground truth")

# Start and end points
plt.plot(self.groundtruth_data[0, 1], self.groundtruth_data[0, 2], 'gx', label="Start point")
plt.plot(self.groundtruth_data[-1, 1], self.groundtruth_data[-1, 2], 'rx', label="End point")

# Landmark ground truth locations and indexes
landmark_xs = []
landmark_ys = []
for location in self.landmark_locations:
    landmark_xs.append(self.landmark_locations[location][0])
    landmark_ys.append(self.landmark_locations[location][1])
    index = self.landmark_indexes[location] + 5
    plt.text(landmark_xs[-1], landmark_ys[-1], str(index), alpha=0.5, fontsize=10)
plt.scatter(landmark_xs, landmark_ys, s=200, c='k', alpha=0.2, marker='*', label='Landmark Locations')

plt.title("Robot Groundtruth and Map")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

### TASK EXAMPLE. Represent landmark measurments

#### Import data into pandas dataframes

In [ ]:
# Build dataframes
groundtruth = build_timeseries(self.groundtruth_data, cols=['stamp','x','y','theta'])
measurements = build_timeseries(self.data, cols=['stamp','type','range_l','bearing_l'])
odometry = measurements[measurements.type == -1].rename(columns={'range_l': 'v', 'bearing_l': 'omega'})
landmarks = measurements[measurements.type != -1]
landmarks = filter_static_landmarks(landmarks, self.barcodes_data)




In [ ]:
groundtruth.head(10)

In [ ]:
odometry.head()

In [ ]:
landmarks.head()

#### Transform landmark measurements to global coordinates
Join dataframes (link observations to particular GT locations)

In [ ]:
data = landmarks.join(groundtruth).dropna()
data.head()

Transform distance and bearing to global coordinates

In [ ]:
range_l = data.range_l
bearing_l = data.bearing_l
x_t = data.x
y_t =  data.y
theta_t = data.theta

x = range_l*np.cos(bearing_l)
y = range_l*np.sin(bearing_l)

data['x_l'] = x_t + x*np.cos(theta_t) - y*np.sin(theta_t)
data['y_l'] = y_t + x*np.sin(theta_t) + y*np.cos(theta_t)
data.head()

#### Represent measurements on top of our known map

In [ ]:
# REPRESENT
# Measurements (New added function)
num_landmarks = len(landmarks.type.unique())
sns.scatterplot(data,x="x_l",y="y_l", hue="type", palette=sns.color_palette("husl", num_landmarks),legend=False)    

# Ground truth data
plt.plot(self.groundtruth_data[:, 1], self.groundtruth_data[:, 2], 'b', label="Robot State Ground truth")

# Start and end points
plt.plot(self.groundtruth_data[0, 1], self.groundtruth_data[0, 2], 'gx', label="Start point")
plt.plot(self.groundtruth_data[-1, 1], self.groundtruth_data[-1, 2], 'rx', label="End point")

# Landmark ground truth locations and indexes
landmark_xs = []
landmark_ys = []
for location in self.landmark_locations:
    landmark_xs.append(self.landmark_locations[location][0])
    landmark_ys.append(self.landmark_locations[location][1])
    index = self.landmark_indexes[location] + 5
    plt.text(landmark_xs[-1], landmark_ys[-1], str(index), alpha=0.5, fontsize=10)
plt.scatter(landmark_xs, landmark_ys, s=200, c='k', alpha=0.2, marker='*', label='Landmark Locations')

# plt.title("Localization with only odometry data")
plt.title("Robot GT, Map and Absolute Measurements")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

### TASK 1. Get dataset specifications

#### Get path length
Sum displacements from the GT

#### Get duration

#### Get number of landmark measurements

#### Get distance between initial and end positions

#### Measurements density 
Mean number of measures per unit of distance

### TASK 2. Iterate over diferent datasets


#### Generate metrics

In [ ]:
datasets = ["../data/MRCLAM_Dataset1",
            "../data/MRCLAM_Dataset2",
            "../data/MRCLAM_Dataset3",
            "../data/MRCLAM_Dataset4"]
robots = ['Robot1',
          'Robot2',
          'Robot3',
          'Robot4']

metrics = pd.Dataframe(columns=['dataset','robot','length','duration','n_landmarks','distance','m_density'])

#for ds in datasets:
    #for rob in robots:
        # Load data
        
        # Get metrics

#### Represent metrics
Use seaborn library (catplot)

In [ ]:
# Represent metrics for all datasets

#### Analize metrics

In [ ]:
# Analize the results and select three datasets with different 'distance','m_density' for the next lab sessions.